In [ ]:
import pandas as pd
import requests
from time import sleep

# Liste des indicateurs à récupérer depuis l'API
indicateurs = [
    "SP.POP.TOTL", "EN.POP.DNST", "SP.POP.GROW", "SP.URB.TOTL.IN.ZS",
    "SP.POP.0014.TO", "SP.POP.65UP.TO", "SP.URB.TOTL", "SH.MED.BEDS.ZS",
    "SH.MED.NUMW.P3", "SP.DYN.LE00.IN", "SP.DYN.IMRT.IN", "SH.STA.MMRT",
    "SH.DYN.NMRT", "SH.DYN.AIDS.ZS", "SH.XPD.PCAP", "SH.XPD.CHEX.GD.ZS",
    "SN.ITK.DEFC.ZS", "SH.STA.OB18.ZS", "SH.XPD.GHED.GE.ZS", "SH.XPD.OOPC.CH.ZS",
    "SH.XPD.PVTD.CH.ZS", "SH.XPD.PVTD.PC.ZS", "GC.DOD.TOTL.GD.ZS", "SH.IMM.MEAS",
    "SH.IMM.POL3", "SH.IMM.IDPT", "SH.STA.ACSN", "SH.H2O.SAFE.ZS", "SH.HOSP.TOTL",
    "SH.MED.EMER.NUM", "SH.HOSP.CTT.SPZ", "SH.SYS.NATL.PRG", "SH.TBS.INCD",
    "SP.POP.65UP.TO.ZS", "SH.XPD.GHED.PC.ZS", "SH.SYS.ACSN"
]

# Récupérer la liste des codes de pays depuis l'API
def get_country_codes():
    url = "http://api.worldbank.org/v2/country?format=json&per_page=300"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Filtrer uniquement les pays (exclude 'income level' and 'region' aggregates)
    country_codes = [
        country['id'] for country in data[1] 
        if country['region']['id'] != 'NA'  # Exclut les agrégats non géographiques
    ]
    return country_codes

# URL de base de l'API
base_url = "http://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json&per_page=500"

def extraire_donnees(indicateur, country_codes, retries=3, backoff_factor=0.3):
    """Récupère les données d'un indicateur depuis l'API et extrait les colonnes nécessaires."""
    dfs = []
    for country in country_codes:
        url = base_url.format(country=country, indicator=indicateur)
        for attempt in range(retries):
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()
                if len(data) > 1:
                    df = pd.DataFrame(data[1])
                    if set(['indicator', 'country', 'date', 'value']).issubset(df.columns):
                        dfs.append(df[['indicator', 'country', 'date', 'value']])
                break  # Si réussi, quitter la boucle de retry
            except (requests.RequestException, ValueError) as e:
                print(f"Erreur lors de la récupération des données: {e}")
                sleep(backoff_factor * (2 ** attempt))  # Attente exponentielle
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Récupération des codes de pays
country_codes = get_country_codes()

# Récupération et concaténation des données pour chaque indicateur
dfs = []
for indicateur in indicateurs:
    df = extraire_donnees(indicateur, country_codes)
    if not df.empty:
        dfs.append(df)

# Concatenation de toutes les données dans un seul DataFrame
donnees_finales = pd.concat(dfs, ignore_index=True)

# Aperçu des données finales
print(donnees_finales.head())

# Sauvegarde en CSV pour analyse ultérieure
donnees_finales.to_csv("donnees_worldbank_tous_pays.csv", index=False)
print("Données extraites et sauvegardées avec succès.")
